In [9]:
import os
import json
import pandas as pd
from urllib.parse import urlparse

data_path = os.path.join("D:/", "data", "drmkc")

In [21]:
filename = "drmkc-scrape_20210218.json"

with open(os.path.join(data_path, filename), 'r') as f:
    data = json.load(f)

In [22]:
for entry in data:
    entry['links'] = [url for url in entry['links'] if "http" in url]
    entry['links'] = list(set([urlparse(url).netloc for url in entry['links']]))
    entry['url'] = urlparse(entry['url']).netloc
    entry['links'] = [url for url in entry['links'] if url != entry['url']]

In [29]:
df = pd.DataFrame.from_records(data)
df = df.explode('links').loc[:, ['url', 'links']]
df = df.drop_duplicates()

In [32]:
outname = "drmkc-nw_20210218.csv"

df.to_csv(os.path.join(data_path, outname), index = False)

In [10]:
domain = urlparse(data[1].get('url')).netloc
print(domain) # --> www.example.test

drmkc.jrc.ec.europa.eu


In [35]:
df.head()

,url,links
0,drmkc.jrc.ec.europa.eu,ec.europa.eu
0,drmkc.jrc.ec.europa.eu,europa.eu
2,drmkc.jrc.ec.europa.eu,www.securityresearch-cou.eu
7,drmkc.jrc.ec.europa.eu,covid-statistics.jrc.ec.europa.eu
7,drmkc.jrc.ec.europa.eu,github.com


In [45]:
filename = "drmkc-scrape_20210218.json"

with open(os.path.join(data_path, filename), 'r') as f:
    data = json.load(f)
    
for entry in data:
    entry['links'] = [url for url in entry['links'] if "http" in url]
    entry['main_links'] = list(set([urlparse(url).netloc for url in entry['links']]))
    entry['main_url'] = urlparse(entry['url']).netloc
    entry['links'] = [url for url in entry['links'] if url != entry['url']]

df = pd.DataFrame.from_records(data)
df = df.explode('links').loc[:, ['url', 'main_url', 'links']]
df = df.drop_duplicates()
df = df.dropna()
df['main_link'] = df['links'].apply(lambda url: urlparse(url).netloc)
df = df.loc[df['main_url'] != df['main_link'], :]

In [46]:
df.head()

,url,main_url,links,main_link
0,https://drmkc.jrc.ec.europa.eu/,drmkc.jrc.ec.europa.eu,https://ec.europa.eu/info/index_en,ec.europa.eu
0,https://drmkc.jrc.ec.europa.eu/,drmkc.jrc.ec.europa.eu,https://europa.eu/european-union/contact/socia...,europa.eu
0,https://drmkc.jrc.ec.europa.eu/,drmkc.jrc.ec.europa.eu,https://ec.europa.eu/knowledge4policy,ec.europa.eu
0,https://drmkc.jrc.ec.europa.eu/,drmkc.jrc.ec.europa.eu,https://ec.europa.eu/info/resources-partners_en,ec.europa.eu
0,https://drmkc.jrc.ec.europa.eu/,drmkc.jrc.ec.europa.eu,https://ec.europa.eu/info/cookies_en,ec.europa.eu


In [47]:
len(list(df['links'].unique()))

126

In [49]:
start_urls = list(df['links'].unique())

In [50]:
outname = "drmkc_links.txt"
with open(os.path.join("..", outname), 'w', encoding = 'utf-8') as f:
    for url in start_urls:
        f.write(url + "\n")
    f.close()
